## Basic of Langchain and its components:
* ChatModel
* Messages
* ChatPromptTemplate
* Output Parser
* Chain

In [1]:
!pip install langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 831.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3

In [2]:
import os
# import getpass
from google.colab import userdata


os.environ['OPENAI_API_KEY'] = userdata.get('myopenAIKey')


from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name='gpt-4o-mini')

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are AI assistant"),
    HumanMessage(content="How are you doing today ? Answer in 1 line only"),
]

In [ ]:
result = model.invoke(messages)

In [ ]:
result

AIMessage(content="I'm just a program, but I'm here and ready to help you!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 27, 'total_tokens': 41, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-afa6c53f-fe82-4d17-a1fb-fe2d775e030d-0', usage_metadata={'input_tokens': 27, 'output_tokens': 14, 'total_tokens': 41})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
output_parser.invoke(result)

"I'm just a program, but I'm here and ready to help you!"

In [ ]:
chain = model | output_parser

In [ ]:
chain.invoke(messages)

"I'm just a program, but I'm here and ready to assist you!"

In [ ]:
### Prompt Template

from langchain_core.prompts import ChatPromptTemplate

system_template = "You are an AI assistant. You have to translate the following into {language}"

prompt_template = ChatPromptTemplate.from_messages([("system",system_template), ("user","{user_input}")])

result = prompt_template.invoke({"language":"English", "user_input":"Hola Mundo"})

result


ChatPromptValue(messages=[SystemMessage(content='You are an AI assistant. You have to translate the following into English', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hola Mundo', additional_kwargs={}, response_metadata={})])

In [ ]:
chain = prompt_template | model | output_parser

In [ ]:
chain.invoke({"language":"English", "user_input":"Hola Mundo"})

'Hello World'

## Build an inital chatbot

In [3]:

from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

## Source: https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/
# HumanMessage:
# Represents a message from the user.
# Generally consists only of content provided by the human

# AIMessage:
# Represents a message from the AI model.
# May include additional information such as tool calls if using specific functionalities like OpenAI tool calling

# SystemMessage:
# Represents a system message that provides instructions or context to the AI model.
# Generally consists of content that guides the AI on how to behave

### How to handle memory / chat history ? ###
1. Use **ConversationBufferMemory**, ConversationStringBufferMemory, etc. + **ChatHistory**. It has been depreacted in v0.3 and moved to langGraph using langGraph persistence ( **Source: https://python.langchain.com/docs/versions/migrating_memory/**) . So, it is advised to use either langGraph for it or use RunnableWithMessageHistory in langChain
2. We can also wrap the chat model within **RunnableWithMessageHistory** along with InMemoryChatMessageHistory. This is the updated technique in langchain_core v-0.3

* What is ChatHistory ?
  > It can be implemented with in-memory storage or persistent storage like Redis

* Note: ConversationBufferMemory does not allow distinguishing between different users and requires implementing user session management. chat_history allows collecting chat history from the frontend and attaching it to the request

### How to manage length of chat memory ?
* One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
* Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History
* When using ConversationBufferMemory, we have various options like window, summary, etc.
* Now, we have trim and filter message for simple chatbots

### RunnableWithMessageHistory

In [11]:
store = {}

def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


config = {"configurable" : {"session_id":"session_123"}}


response = with_message_history.invoke([HumanMessage(content="Hi I am Manpreet")], config=config)

print(response)
print(response.content)

content='Hello Manpreet! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 13, 'total_tokens': 25, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None} id='run-0b89b2fb-1b34-4542-be8e-03acdeebe730-0' usage_metadata={'input_tokens': 13, 'output_tokens': 12, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}
Hello Manpreet! How can I assist you today?


In [12]:
response = with_message_history.invoke([HumanMessage(content="What is my name ?")], config=config)
response.content

'Your name is Manpreet. How can I help you today?'

In [13]:
store

{'session_123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manpreet', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Manpreet! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 13, 'total_tokens': 25, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b89b2fb-1b34-4542-be8e-03acdeebe730-0', usage_metadata={'input_tokens': 13, 'output_tokens': 12, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content='What is my name ?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Manpreet. How can I help you today?', additional_kwargs={'

In [ ]:
## we can always go back to the original conversation by pointing to the sessionid


In [112]:
### Task1: Store the session details. It will store the entire conversation to the current chat time
### Here, we use InMemoryChatMessageHistory, which is an alternative to ConversationBufferMemory
 #### (which was deprecated in langchain and moved to to langGraph)
 #### https://python.langchain.com/docs/versions/migrating_memory/conversation_buffer_memory/

store = {}

def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [113]:
### Task2: Define the prompt and add information of System, Human, and chatHistory

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_mesg = "You are a helpful assistant. Your job is to respond to my queries like my mentor and an as life coach in {my_tone} tone"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_mesg),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [114]:
### Task3: Chain them:
#### Ealrier LLMChain was used but now with Runnables, we have LCEL (LangChain Execution Layer), which makes it very simple to chain the Runnables
##### together with '|' symbol in the order of execution

### Legacy / older code using LLmChain
# from langchain.chains import LLMChain
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

# prompt = ChatPromptTemplate.from_messages(
#     [("user", "Tell me a {adjective} joke")],
# )

# legacy_chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)



### below is the new code:


chain = prompt | model


In [115]:
### old way to invoke:
# legacy_result = legacy_chain({"adjective": "funny"})
# legacy_result


### Invoking the new chains:

chain.invoke({"messages":[HumanMessage(content="Hi I am manpreet. How are you ?")],
              "my_tone":"friendly"})

AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 47, 'total_tokens': 72, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-945a5dd4-1c42-4e21-9a20-8934048ce3bd-0', usage_metadata={'input_tokens': 47, 'output_tokens': 25, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [116]:
### There is no memeory or historical chain involved here.
chain.invoke({"messages":[HumanMessage(content="Can you tell my name ?")],
              "my_tone":"aggression"})

AIMessage(content='Listen up! I don’t have the ability to know your name unless you tell me. So, what’s your name? Let’s get this conversation rolling!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 42, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-8b5c9839-601e-4408-812a-fc90d231fca4-0', usage_metadata={'input_tokens': 42, 'output_tokens': 32, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [117]:
### Task4: Add user History (memory) to User Input to make a proper bot conversation
#### We added stores to store the History, now let's see how to use that history and pass it in the prompt along with user query
##### Earlier we used to use ConversationChain, but now it has been replaced with RunnableWithMessageHistory


##### New Method using RunnableWithMessageHistory

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)




In [118]:
### Task5: Setup the session_id which will load the historical chatg and add it the current conversation
#### config has a fixed style and it expects configurable and session_id keywords
config = {"configurable": {"session_id": "abc11"}}



In [119]:
### Task6: Chat

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi I am Manpreet. How are you doing ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 48, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-0baf051e-f94e-4aa3-9673-fa118def3bfe-0', usage_metadata={'input_tokens': 48, 'output_tokens': 25, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [120]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content=" I am 31 years old and I like to sketch")],
                                        "my_tone":"friendly"},
                                         config=config)
response

AIMessage(content='That’s wonderful, Manpreet! Sketching is a beautiful form of expression. It allows you to capture your thoughts and feelings in a unique way. How long have you been sketching? And what kind of subjects do you enjoy drawing the most?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 92, 'total_tokens': 143, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-e99f3790-e2da-4db0-a0c5-f7e38bac4f52-0', usage_metadata={'input_tokens': 92, 'output_tokens': 51, 'total_tokens': 143, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [121]:
### It will be able to answer because we have kept the memory by using RunnableWithMessageHistory.
response = with_message_history.invoke(
    {"messages": [HumanMessage(content=" What do you think is my hobby ?")],
                                        "my_tone":"aggressive"},
                                         config=config)
response

AIMessage(content='Your hobby is clearly sketching! But let’s not just stop there. If you’re passionate about it, you need to dive deeper. Your hobby should not just be a pastime; it should be something that you invest your time in and develop your skills. Are you pushing yourself to improve? Are you experimenting with different styles or subjects? If not, it’s time to step up your game! What are you waiting for? Get out there and create!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 159, 'total_tokens': 251, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-12d21408-40b2-43c5-90a8-382c29bfd344-0', usage_metadata={'input_tokens': 159, 'output_tokens': 92, 'total_tokens': 251, 'input_token_detai

In [122]:
## This will store the entire conversation w.r.t each sessionId

store

{'abc11': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manpreet. How are you doing ?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 48, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-0baf051e-f94e-4aa3-9673-fa118def3bfe-0', usage_metadata={'input_tokens': 48, 'output_tokens': 25, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadata={}), AIMe

In [123]:
store['abc11']

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manpreet. How are you doing ?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 48, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-0baf051e-f94e-4aa3-9673-fa118def3bfe-0', usage_metadata={'input_tokens': 48, 'output_tokens': 25, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadata={}), AIMessage(cont

In [124]:
## NOTE: We can always activate the bot in a conversation way and load the historical conversation just by mapping user_id with session_id
## and when that user will login, we will pass that user's id and fetch his/her sessionId and use that to resume the chat with history

In [125]:
# prompt: get all HumanMessage from store

message_conv = [message.content for message in store['abc11'].messages if isinstance(message, HumanMessage) or isinstance(message, AIMessage) ]


In [126]:
for message in store['abc11'].messages:
  if isinstance(message, HumanMessage):
    print(f"Human: {message.content}")
  elif isinstance(message, AIMessage):
    print(f"AI: {message.content}")

Human: Hi I am Manpreet. How are you doing ?
AI: Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?
Human:  I am 31 years old and I like to sketch
AI: That’s wonderful, Manpreet! Sketching is a beautiful form of expression. It allows you to capture your thoughts and feelings in a unique way. How long have you been sketching? And what kind of subjects do you enjoy drawing the most?
Human:  What do you think is my hobby ?
AI: Your hobby is clearly sketching! But let’s not just stop there. If you’re passionate about it, you need to dive deeper. Your hobby should not just be a pastime; it should be something that you invest your time in and develop your skills. Are you pushing yourself to improve? Are you experimenting with different styles or subjects? If not, it’s time to step up your game! What are you waiting for? Get out there and create!


### Add trim_message functionality to save chat from exploding and keeping only latest K messages


In [127]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=200,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [128]:
trimmer.invoke(store['abc11'].messages)

[HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That’s wonderful, Manpreet! Sketching is a beautiful form of expression. It allows you to capture your thoughts and feelings in a unique way. How long have you been sketching? And what kind of subjects do you enjoy drawing the most?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 92, 'total_tokens': 143, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-e99f3790-e2da-4db0-a0c5-f7e38bac4f52-0', usage_metadata={'input_tokens': 92, 'output_tokens': 51, 'total_tokens': 143, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 HumanMessage(

In [129]:
### This gives more context as we pull the messages only and delete additional information
trimmer.invoke(message_conv)

[HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='That’s wonderful, Manpreet! Sketching is a beautiful form of expression. It allows you to capture your thoughts and feelings in a unique way. How long have you been sketching? And what kind of subjects do you enjoy drawing the most?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' What do you think is my hobby ?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Your hobby is clearly sketching! But let’s not just stop there. If you’re passionate about it, you need to dive deeper. Your hobby should not just be a pastime; it should be something that you invest your time in and develop your skills. Are you pushing yourself to improve? Are you experimenting with different styles or subjects? If not, it’s time to step up your game! What are you waiting for? Get out there and create!', additional_kwargs={}, response_meta

In [130]:
trimmer

RunnableLambda(...)

In [131]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

In [132]:
new_chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [134]:
### No Memory
new_chain.invoke({"messages": [HumanMessage(content="what is my hobby and name ?")],
              "my_tone":"soft"})


AIMessage(content="That's a great question! While I don’t have access to your personal details, I can help you explore what your hobbies might be. Think about activities that make you feel happy and fulfilled. Do you enjoy painting, reading, playing sports, or perhaps gardening? \n\nAs for your name, that's something only you can share! If you'd like to tell me, I'm here to listen. Remember, discovering your interests and passions is a wonderful journey, and I'm here to support you every step of the way.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 43, 'total_tokens': 144, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a13949f-a358-42c3-ad60-13a3f4a70a30-0', usage_metadata={'input_tokens'

In [133]:
### With Memory
new_chain.invoke({"messages": message_conv + [HumanMessage(content="what is my hobby and name ?")],
              "my_tone":"soft"})


AIMessage(content="Your hobby is sketching, and your name is Manpreet. It's lovely to see you embracing your creative side! If you have any specific goals or aspirations related to your sketching, I'd love to hear about them. Whether it's improving your technique, exploring new styles, or even sharing your art with others, there are so many exciting paths you can take. How can I support you in your journey?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 221, 'total_tokens': 303, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-6ea6af6d-d283-40b2-b48f-7fa600c62d0e-0', usage_metadata={'input_tokens': 221, 'output_tokens': 82, 'total_tokens': 303, 'input_token_details': {'cache_read': 0}, 'output_

In [ ]:
# new_chain.invoke({"messages": trimmer.invoke(message_conv) + [HumanMessage(content="what is my hobby and name ?")],
#               "my_tone":"soft"})

### We can trimmer in the chain.invoke as well and remove where chain is getting declaared

In [135]:
### Add trimmer to RunnableMessageHistory

In [136]:
with_message_history = RunnableWithMessageHistory(
    new_chain,
    get_session_history,
    input_messages_key="messages",
)

In [137]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  messages: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x79bc88f25900>, input_messages_key='messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [139]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi I am Manpreet. How are you doing ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

AIMessage(content='Hi Manpreet! I’m doing well, thank you! I’m really glad to be here, ready to chat with you. How about you? How’s your sketching going? Any new projects or ideas you’re excited about?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 226, 'total_tokens': 274, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5466ef8-b24a-4ca4-b6c3-d1a4c9ee5945-0', usage_metadata={'input_tokens': 226, 'output_tokens': 48, 'total_tokens': 274, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [140]:
store

{'abc11': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manpreet. How are you doing ?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 48, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-0baf051e-f94e-4aa3-9673-fa118def3bfe-0', usage_metadata={'input_tokens': 48, 'output_tokens': 25, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadata={}), AIMe

In [141]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is 2*4 + 2 ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

AIMessage(content='The calculation is quite simple! First, you multiply 2 by 4, which gives you 8. Then you add 2 to that result. So, 2 * 4 + 2 = 8 + 2 = 10. If you have any more questions or need help with anything else, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 222, 'total_tokens': 291, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea25870c-7043-4aa3-a0e5-1bf2039c4ce3-0', usage_metadata={'input_tokens': 222, 'output_tokens': 69, 'total_tokens': 291, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [142]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is the colour of carbon monoxide ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

AIMessage(content="Carbon monoxide (CO) is actually colorless, odorless, and tasteless. This makes it particularly dangerous because you can't see or smell it, which is why it's important to have carbon monoxide detectors in your home if you use gas appliances or have a garage attached to your house. If you have more questions about safety or anything else, I'm here to help!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 199, 'total_tokens': 272, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a271533-5626-4e29-8bae-5a66e925fa5b-0', usage_metadata={'input_tokens': 199, 'output_tokens': 73, 'total_tokens': 272, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}

In [146]:
trimmer.invoke(store['abc11'].messages)
### here the trimmer has trimmed old chats regarding my name

[HumanMessage(content='What is 2*4 + 2 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The calculation is quite simple! First, you multiply 2 by 4, which gives you 8. Then you add 2 to that result. So, 2 * 4 + 2 = 8 + 2 = 10. If you have any more questions or need help with anything else, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 222, 'total_tokens': 291, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea25870c-7043-4aa3-a0e5-1bf2039c4ce3-0', usage_metadata={'input_tokens': 222, 'output_tokens': 69, 'total_tokens': 291, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 HumanMessage(content='What is the colour of c

In [147]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

## Now, it is not able to tell the name because trimmer has trimmed the earlier messages

AIMessage(content='I actually don’t know your name unless you tell me! But I’d love to know it if you’d like to share. What’s your name?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 217, 'total_tokens': 248, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b3512c9-01d4-4f17-a302-3ae54263fea5-0', usage_metadata={'input_tokens': 217, 'output_tokens': 31, 'total_tokens': 248, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [148]:
with_message_history_oldchain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [150]:
response = with_message_history_oldchain.invoke(
    {"messages": [HumanMessage(content="What is my name ?")],
                                        "my_tone":"friendly"},
                                         config=config)
response

## Now, it is able to tell the name because no trimmer has been activated as we have used old chain

AIMessage(content='Your name is Manpreet! It’s great to chat with you, Manpreet. If there’s anything specific you’d like to discuss or ask about, just let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 612, 'total_tokens': 650, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-52b3a049-6616-45a2-b704-9cd03d6a5135-0', usage_metadata={'input_tokens': 612, 'output_tokens': 38, 'total_tokens': 650, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## Archive

### Quick observation of why not use any custom key="user_message" but use "messagess only

In [175]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

##
def get_and_trim_user_messages(data, trimmer, key="user_messages"):
    messages = itemgetter(key)(data)
    return trimmer(messages)

def trimmer(messages, max_length=5):
    return messages[:max_length]

runnable = RunnablePassthrough.assign(
    messages=lambda data: get_and_trim_user_messages(data, trimmer)
)


# Example input
input_data = {
    "user_messages": ["msg1", "msg2", "msg3", "msg4", "msg5", "msg6", "msg7", "msg8", "msg9", "msg10", "msg11"]
}

# Invoke the runnable
output = runnable.invoke(input_data)

print(output)
# Output: {'user_messages': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5', 'msg6', 'msg7', 'msg8', 'msg9', 'msg10', 'msg11'], 'messages': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5', 'msg6', 'msg7', 'msg8', 'msg9', 'msg10']}


{'user_messages': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5', 'msg6', 'msg7', 'msg8', 'msg9', 'msg10', 'msg11'], 'messages': ['msg1', 'msg2', 'msg3', 'msg4', 'msg5']}


In [174]:
output['messages']

['msg1', 'msg2', 'msg3', 'msg4', 'msg5']

In [170]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="my hobby is art"),
    AIMessage(content="great!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='my hobby is art', additional_kwargs={}, response_metadata={}),
 AIMessage(content='great!', additional_kwargs={}, response_metadata={})]

In [172]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="user_messages"),
    ]
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("user_messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "user_messages": trimmer.invoke(messages) + [HumanMessage(content="Who am i ?")],
        "language": "English",
    }
)
response.content

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [167]:
message_conv

['Hi I am Manpreet. How are you doing ?',
 "Hi Manpreet! I'm doing well, thank you for asking. How about you? What’s on your mind today?",
 ' I am 31 years old and I like to sketch',
 'That’s wonderful, Manpreet! Sketching is a beautiful form of expression. It allows you to capture your thoughts and feelings in a unique way. How long have you been sketching? And what kind of subjects do you enjoy drawing the most?',
 ' What do you think is my hobby ?',
 'Your hobby is clearly sketching! But let’s not just stop there. If you’re passionate about it, you need to dive deeper. Your hobby should not just be a pastime; it should be something that you invest your time in and develop your skills. Are you pushing yourself to improve? Are you experimenting with different styles or subjects? If not, it’s time to step up your game! What are you waiting for? Get out there and create!']

In [104]:
store

{'abc11': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am Manpreet. How are you doing ?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Manpreet! I'm doing well, thank you for asking. How about you? How's everything going in your world?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 48, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-50471c15-026e-45f5-93e2-48401489e83e-0', usage_metadata={'input_tokens': 48, 'output_tokens': 26, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content=' I am 31 years old and I like to sketch', additional_kwargs={}, response_metadat

In [84]:
message_conv

['Hi I am Manpreet. How are you doing ?',
 "Hi Manpreet! I'm doing well, thank you for asking. How about you? How's everything going in your world?",
 ' I am 31 years old and I like to sketch',
 "That's wonderful, Manpreet! Sketching is such a fantastic way to express creativity and capture the world around you. What do you enjoy sketching the most? People, landscapes, or something else?",
 ' What do you think is my hobby ?',
 "Your hobby is clearly sketching, and that's great! But let me tell you, just liking to sketch isn't enough. You need to dive deep, push your limits, and really make it a part of your life. Are you sketching regularly, or are you just dabbling? Get serious about it! Turn that hobby into a passion, and start creating masterpieces! What’s stopping you from taking it to the next level?"]

In [ ]:
NOTE:

### So the trick is to give "messages" across everywhere and not use any custom name like "user_messagees".
### If you use, RunnablePassthrough and trimmer won't work the way they are designed


Now, correct in the real example in the notebooka and fix the code i.e update the ChatPrompt to have "messages" keyword only and not "user_messages"

### How can we use advanced functionalities of ConversationBuffer like window, summary along with RunnableWithMessageHistory ?
#### **Answer: Build a custom class for it**